In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Burn])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 50

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, owner, tick_lower, tick_upper = evt["topics"]
    args = Conversion().decode_data(evt["data"])
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['owner'] = Conversion().convert_uint256_hex_string_to_address(owner)
    event['details']['tick_lower'] = Conversion().convert_uint256_string_to_int(tick_lower, signed=True)
    event['details']['tick_upper'] = Conversion().convert_uint256_string_to_int(tick_upper, signed=True)    
    if(len(args) == 4):
        event['details']['liquidity_amount'] = Conversion().convert_int256_bytes_to_int(args[1])
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[2])
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[3])
    events[k] = event
    
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Swap at block:60,353,029 tx:0x647074ee784f26d7f5aa0e8b218ded481b05d85d12f3ecb6f778d4ff3f537192
Swap at block:60,353,030 tx:0xdd5646266194a8242b7d0a8f68422c54472d55200878c8a093344e654c35bbcb
Swap at block:60,353,030 tx:0xdd5646266194a8242b7d0a8f68422c54472d55200878c8a093344e654c35bbcb
Swap at block:60,353,030 tx:0xdd5646266194a8242b7d0a8f68422c54472d55200878c8a093344e654c35bbcb
Swap at block:60,353,030 tx:0xdd5646266194a8242b7d0a8f68422c54472d55200878c8a093344e654c35bbcb
Swap at block:60,353,030 tx:0x0246bcbf839e51b2d29a1dd62350b33c71e8b0826332e43db9529048d35c3c8f
Swap at block:60,353,037 tx:0x313c79ea1a360fa21931dadeecbd55a0f43391db1e7379647e13bb4d042997f1
Swap at block:60,353,039 tx:0x57b0b5b71310524c2c717e6ab574658caaf8db8dd429c4880e42f1dd511c1d16
Swap at block:60,353,040 tx:0xd5d4ea7068802dbb5106e4b60e84d1e0f2e499f1f9219deb6efb7bd030f82de2
Swap at block:60,353,046 tx:0x6383427319d91fe515ca663f1a0733bf45b6714818b0dca64931c1a79fcf37f3
Swap at block:60,353,055 tx:0x5319a27fa973f38b18c1

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,burn,uniswap_v3,0x32fae204835e08b9374493d6b4628fd1f87dd045,0x647074ee784f26d7f5aa0e8b218ded481b05d85d12f3...,60353029,1723148953,{'web3_type': <class 'web3._utils.datatypes.Bu...
1,polygon,uniswapv3pool,burn,uniswap_v3,0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7,0xdd5646266194a8242b7d0a8f68422c54472d55200878...,60353030,1723148955,{'web3_type': <class 'web3._utils.datatypes.Bu...
2,polygon,uniswapv3pool,burn,uniswap_v3,0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7,0xdd5646266194a8242b7d0a8f68422c54472d55200878...,60353030,1723148955,{'web3_type': <class 'web3._utils.datatypes.Bu...
3,polygon,uniswapv3pool,burn,uniswap_v3,0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7,0xdd5646266194a8242b7d0a8f68422c54472d55200878...,60353030,1723148955,{'web3_type': <class 'web3._utils.datatypes.Bu...
4,polygon,uniswapv3pool,burn,uniswap_v3,0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7,0xdd5646266194a8242b7d0a8f68422c54472d55200878...,60353030,1723148955,{'web3_type': <class 'web3._utils.datatypes.Bu...
5,polygon,uniswapv3pool,burn,uniswap_v3,0xabebc245a9a47166ecd10933d43817c8ef6fb825,0x0246bcbf839e51b2d29a1dd62350b33c71e8b0826332...,60353030,1723148955,{'web3_type': <class 'web3._utils.datatypes.Bu...
6,polygon,uniswapv3pool,burn,uniswap_v3,0x72d6021170f608a40d2df1a582afa92e5ea8c776,0x313c79ea1a360fa21931dadeecbd55a0f43391db1e73...,60353037,1723148969,{'web3_type': <class 'web3._utils.datatypes.Bu...
7,polygon,uniswapv3pool,burn,uniswap_v3,0xb3866eb993e1aef93f219c3da0a71c3f11becbf2,0x57b0b5b71310524c2c717e6ab574658caaf8db8dd429...,60353039,1723148973,{'web3_type': <class 'web3._utils.datatypes.Bu...
8,polygon,uniswapv3pool,burn,uniswap_v3,0xc4c06c9a239f94fc0a1d3e04d23c159ebe8316f1,0xd5d4ea7068802dbb5106e4b60e84d1e0f2e499f1f921...,60353040,1723148977,{'web3_type': <class 'web3._utils.datatypes.Bu...
9,polygon,uniswapv3pool,burn,uniswap_v3,0x72d6021170f608a40d2df1a582afa92e5ea8c776,0x6383427319d91fe515ca663f1a0733bf45b6714818b0...,60353046,1723148989,{'web3_type': <class 'web3._utils.datatypes.Bu...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platform': 'uniswap_v3',
  'address': '0x32fae204835e08b9374493d6b4628fd1f87dd045',
  'tx_hash': '0x647074ee784f26d7f5aa0e8b218ded481b05d85d12f3ecb6f778d4ff3f537192',
  'blk_num': 60353029,
  'timestamp': 1723148953,
  'details': {'web3_type': web3._utils.datatypes.Burn,
   'owner': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
   'tick_lower': 63080,
   'tick_upper': 64870}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platform': 'uniswap_v3',
  'address': '0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7',
  'tx_hash': '0xdd5646266194a8242b7d0a8f68422c54472d55200878c8a093344e654c35bbcb',
  'blk_num': 60353030,
  'timestamp': 1723148955,
  'details': {'web3_type': web3._utils.datatypes.Burn,
   'owner': '0xBB02DB91f2a6917F73acD39659FD51488B005063',
   'tick_lower': -286860,
   'tick_upper': -283360}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platf